# 2025-09-06

In [ ]:
# yield is like return, but it doesn't break out of the function!

import time

def countdown(n):
    while n > 0:
        yield n # this turns the function into a generator function
        n -= 1

#run_this = True
run_this = False
if run_this:
    c = countdown(10)
    for i in c:
        print(i)
        time.sleep(1)
    print("LIFTOFF")

# obvious issue with the above, using sleep, is that it doesn't actually fire once per second
# like delay in arduino, we need to replace it with the equivalent of millis


# __next__(self) is the method that when included in an object tells python that it can be iterated over 
# (used in loops and such)
# it's what the loop actually executes under the hood
c = countdown(3)
print(next(c)) # 3
print(next(c)) # 2
print(next(c)) # 1
try:
    print(next(c))
except Exception as e:
    print(f"Failed due to: {repr(e)}") # just printing `e` doesn't actually print "StopIteration", because that exception doesn't include a message, but it does have a __repr__ method (no __str__, though which is what happens when we run print(f"{e)") same as print(str(e))


# Background Timer with Interrupts
Like Arduino interrupts, we want a timer that runs independently and can interrupt other code execution.

In [ ]:
import time
import threading
from typing import Callable, Optional

class BackgroundTimer:
    """Arduino-style timer that runs in background and can interrupt other code"""
    
    def __init__(self, total_seconds: int, interval_seconds: int = 10):
        self.total_seconds = total_seconds
        self.interval_seconds = interval_seconds
        self.remaining = total_seconds
        self.running = False
        self.thread = None
        self.interrupt_flag = threading.Event()  # Like Arduino interrupt flag
        self.last_result = None
        
        # Custom thresholds and messages
        self.thresholds = {
            60: "Hurry up, half-time!",
            30: "30 seconds remaining!",
            10: "Final countdown!",
            5: "Almost done!",
            0: "TIME'S UP!"
        }
    
    def start(self):
        """Start the background timer"""
        if not self.running:
            self.running = True
            self.interrupt_flag.clear()
            self.thread = threading.Thread(target=self._timer_loop, daemon=True)
            self.thread.start()
            print(f"⏰ Timer started: {self.total_seconds}s countdown")
    
    def stop(self):
        """Stop the background timer"""
        self.running = False
        if self.thread:
            self.thread.join()
        print("⏹️ Timer stopped")
    
    def _timer_loop(self):
        """Internal timer loop - runs in background thread"""
        start_time = time.perf_counter()
        last_interval_print = 0
        
        while self.running and self.remaining > 0:
            elapsed = time.perf_counter() - start_time
            self.remaining = max(0, self.total_seconds - int(elapsed))
            
            # Print every interval
            current_interval = (self.total_seconds - self.remaining) // self.interval_seconds
            if current_interval > last_interval_print:
                print(f"\n⏱️ Time remaining: {self.remaining}s")
                last_interval_print = current_interval
                self.interrupt_flag.set()  # Signal interrupt
            
            # Check threshold messages
            if self.remaining in self.thresholds:
                print(f"\n🚨 {self.thresholds[self.remaining]}")
                self.interrupt_flag.set()  # Signal interrupt
                time.sleep(1)  # Prevent duplicate messages
            
            time.sleep(0.1)  # Check every 100ms
        
        if self.remaining <= 0:
            print(f"\n⏰ {self.thresholds[0]}")
            self.interrupt_flag.set()
        
        self.running = False
    
    def check_interrupt(self) -> bool:
        """Check if timer has triggered an interrupt (like Arduino interrupts)"""
        if self.interrupt_flag.is_set():
            self.interrupt_flag.clear()
            return True
        return False
    
    def is_time_up(self) -> bool:
        """Check if time is completely up"""
        return self.remaining <= 0 and not self.running

# Test the background timer
timer = BackgroundTimer(total_seconds=120, interval_seconds=10)
print("Timer created but not started yet...")

In [ ]:
def fibonacci_generator():
    """Infinite fibonacci generator - our 'main' code that runs continuously"""
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

def run_with_timer_interrupts():
    """Main function that runs fibonacci calculation with timer interrupts"""
    
    # Start the timer
    timer = BackgroundTimer(total_seconds=15, interval_seconds=3)  # Shorter for demo
    timer.start()
    
    # Start our "main" computation
    fib_gen = fibonacci_generator()
    iteration_count = 0
    last_fib = 0
    
    print("🔢 Starting fibonacci calculation...")
    print("(Timer will interrupt every 3 seconds)")
    
    try:
        while not timer.is_time_up():
            # Do some work (calculate next fibonacci)
            last_fib = next(fib_gen)
            iteration_count += 1
            
            # Check for timer interrupt (like Arduino interrupt handling)
            if timer.check_interrupt():
                print(f"💡 INTERRUPT! Current fibonacci: {last_fib} (iteration {iteration_count})")
                print("   Continuing calculation...")
            
            # Small delay to make it visible (remove for real work)
            time.sleep(0.01)
            
            # Print progress occasionally
            if iteration_count % 500 == 0:
                print(f"📊 Progress: fibonacci #{iteration_count} = {last_fib}")
        
        print(f"\n🏁 Final result: fibonacci #{iteration_count} = {last_fib}")
        print("🛑 Main loop terminated - timer finished!")
        
    except KeyboardInterrupt:
        print(f"\n⚠️ Interrupted by user! Last result: {last_fib}")
    finally:
        timer.stop()

# Run the demo
run_with_timer_interrupts()

## Alternative: Signal-Style Interrupts
For even more Arduino-like behavior, here's a simpler approach using callbacks:

In [ ]:
import threading

class SimpleTimer:
    """Simple callback-based timer - even more like Arduino interrupts"""
    
    def __init__(self, total_seconds: int, callback_interval: int = 10):
        self.total_seconds = total_seconds
        self.callback_interval = callback_interval
        self.start_time = None
        self.running = False
        self.shared_data = {"last_result": None, "interrupt_count": 0, "time_up": False}
    
    def start(self, main_loop_function):
        """Start timer and run main loop with interrupts"""
        self.start_time = time.perf_counter()
        self.running = True
        self.shared_data["interrupt_count"] = 0
        self.shared_data["time_up"] = False
        
        print(f"🚀 Starting {self.total_seconds}s timer with {self.callback_interval}s interrupts")
        
        # Start background timer thread
        timer_thread = threading.Thread(target=self._timer_thread, daemon=True)
        timer_thread.start()
        
        # Run main loop
        try:
            main_loop_function(self.shared_data)
        finally:
            self.running = False
    
    def _timer_thread(self):
        """Background thread that triggers interrupts"""
        last_interrupt = 0
        
        while self.running:
            elapsed = time.perf_counter() - self.start_time
            remaining = max(0, self.total_seconds - elapsed)
            
            # Check for interval interrupts
            current_interval = int(elapsed // self.callback_interval)
            if current_interval > last_interrupt:
                last_interrupt = current_interval
                self.shared_data["interrupt_count"] += 1
                
                # This is the "interrupt" - print status
                print(f"\n⚡ TIMER INTERRUPT #{self.shared_data['interrupt_count']}")
                print(f"   Time remaining: {remaining:.1f}s")
                print(f"   Last result: {self.shared_data['last_result']}")
                
                # Threshold messages
                if remaining <= 10:
                    print("   🔥 FINAL COUNTDOWN!")
                elif remaining <= 30:
                    print("   ⚠️ 30 seconds or less!")
                elif remaining <= 60:
                    print("   ⏰ Half time!")
            
            # Check if time is up
            if remaining <= 0:
                print(f"\n🏁 TIME'S UP! Final result: {self.shared_data['last_result']}")
                self.shared_data["time_up"] = True  # Signal main loop to stop
                self.running = False
                break
                
            time.sleep(0.1)

def my_main_loop(shared_data):
    """Main computation loop - calculates fibonacci while timer interrupts"""
    fib_gen = fibonacci_generator()
    count = 0
    
    # Main loop now checks if time is up
    while not shared_data["time_up"]:
        # Do the main work
        result = next(fib_gen)
        count += 1
        
        # Update shared data (like global variables in Arduino)
        shared_data["last_result"] = f"fib[{count}] = {result}"
        
        # Print progress occasionally
        if count % 5000 == 0:
            print(f"📈 Computing... {shared_data['last_result']}")
        
        # Small delay to make demo visible
        time.sleep(0.001)
    
    print("🛑 Main loop stopped - timer finished!")

# Demo the simple timer
timer = SimpleTimer(total_seconds=20, callback_interval=3)
print("Created timer. Call timer.start(my_main_loop) to begin!")

In [ ]:
# Run the simple timer demo
timer.start(my_main_loop)

## Arduino Comparison

This Python approach mimics Arduino timer interrupts:

**Arduino:**
```cpp
void setup() {
    // Setup timer interrupt every 1 second
    Timer1.initialize(1000000); // 1 second
    Timer1.attachInterrupt(timerISR);
}

void timerISR() {
    // Interrupt service routine
    Serial.println("Timer interrupt!");
    timeRemaining--;
}

void loop() {
    // Main code runs continuously
    fibonacci_calculation();
}
```

**Python equivalent:**
- `threading.Thread` = Arduino timer interrupt
- `shared_data` = global variables accessible in ISR
- `check_interrupt()` = checking interrupt flags
- Main loop = Arduino `loop()` function

The key insight: **The timer runs independently and can interrupt/communicate with your main code**, just like hardware interrupts!

## Summary: Background Timer Patterns

You now have **two main approaches** for background timers that interrupt other code:

### 1. **Event Flag Pattern** (`BackgroundTimer`)
- Main code checks `timer.check_interrupt()` periodically
- More control over when interrupts are handled
- Like Arduino polling interrupt flags

### 2. **Callback Pattern** (`SimpleTimer`) 
- Timer directly prints/executes interrupt code
- Main code just runs, timer handles itself
- More like true hardware interrupts

**Key Concepts:**
- `threading.Thread(daemon=True)` = background timer
- `threading.Event()` = interrupt flag signaling
- Shared data structures = global variables in Arduino
- `time.perf_counter()` = precise timing like `millis()`

Both patterns let your main code run continuously while the timer operates independently in the background!

## ✅ Fixed: Proper Timer Termination

**The Problem:** The main loop was running `while True:` with no exit condition.

**The Solution:** Added shared state communication:

1. **Timer sets flag**: `shared_data["time_up"] = True` when timer expires
2. **Main loop checks flag**: `while not shared_data["time_up"]:` instead of `while True:`
3. **Clean termination**: Main loop exits when timer finishes

This is exactly like Arduino global variables that both the main loop and interrupt service routine can access!

**Arduino equivalent:**
```cpp
volatile bool timeUp = false;  // Global variable

void timerISR() {
    if (timeRemaining <= 0) {
        timeUp = true;  // Set flag in interrupt
    }
}

void loop() {
    while (!timeUp) {  // Check flag in main loop
        // Do work
        fibonacci_calculation();
    }
    Serial.println("Timer finished!");
}
```